In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
trainmain=pd.read_csv("C:\\Users\kaushik\Desktop\ML\independence day hackathon\\train.csv")
testmain=pd.read_csv("C:\\Users\kaushik\Desktop\ML\independence day hackathon\\test.csv")

In [3]:
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns
    
def uniquevalues(df):
    for col in df.columns:
        if df[col].dtype=='O':
            print("The unique values of {0} is {1}".format(col,df[col].nunique()))
            
# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category,drop_first=True)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns    

def reduce_mem_usage(props):
    start_mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage of properties dataframe is :",start_mem_usg," MB")
    NAlist = [] # Keeps track of columns that have missing values filled in. 
    for col in props.columns:
        if props[col].dtype != object:  # Exclude strings
            
            # Print current column type
            print("******************************")
            print("Column: ",col)
            print("dtype before: ",props[col].dtype)
            
            # make variables for Int, max and min
            IsInt = False
            mx = props[col].max()
            mn = props[col].min()
            
            # Integer does not support NA, therefore, NA needs to be filled
            if not np.isfinite(props[col]).all(): 
                NAlist.append(col)
                props[col].fillna(mn-1,inplace=True)  
                   
            # test if column can be converted to an integer
            asint = props[col].fillna(0).astype(np.int64)
            result = (props[col] - asint)
            result = result.sum()
            if result > -0.01 and result < 0.01:
                IsInt = True

            
            # Make Integer/unsigned Integer datatypes
            if IsInt:
                if mn >= 0:
                    if mx < 255:
                        props[col] = props[col].astype(np.uint8)
                    elif mx < 65535:
                        props[col] = props[col].astype(np.uint16)
                    elif mx < 4294967295:
                        props[col] = props[col].astype(np.uint32)
                    else:
                        props[col] = props[col].astype(np.uint64)
                else:
                    if mn > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        props[col] = props[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        props[col] = props[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        props[col] = props[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        props[col] = props[col].astype(np.int64)    
            
            # Make float datatypes 32 bit
            else:
                props[col] = props[col].astype(np.float32)
            
            # Print new column type
            print("dtype after: ",props[col].dtype)
            print("******************************")
    
    # Print final result
    print("___MEMORY USAGE AFTER COMPLETION:___")
    mem_usg = props.memory_usage().sum() / 1024**2 
    print("Memory usage is: ",mem_usg," MB")
    print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
    return props, NAlist

def dateformatchange(date):
    Month=date.split('-')[0]
    Year="20"+ date.split('-')[1]
    monthnum=d.get(Month)
    newdate= "0" + str(monthnum) + "/" + Year
    return newdate

def checkmvalue(df):
    allm=['m1','m2','m3','m4','m5','m6','m7','m8','m9','m10','m11']
    j=0
    for i in allm:
        if df[i]>0:
            j=1
    return j  
def checknummissed(df):
    allm=['m1','m2','m3','m4','m5','m6','m7','m8','m9','m10','m11']
    j=0
    for i in allm:
        if df[i]>0:
            j=j+1
    return j 

def checkmaxmissed(df):
    allm=['m1','m2','m3','m4','m5','m6','m7','m8','m9','m10','m11']
    j=df[allm].max()
    return j

In [4]:
df=trainmain.copy()
df_test=testmain.copy()
df.drop('loan_id',axis=1,inplace=True)
df_test.drop('loan_id',axis=1,inplace=True)
df_test['origination_date'] =pd.to_datetime(df_test['origination_date']).dt.strftime('%Y-%d-%m')
d = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5,'Jun':6,'Jul':7,'Aug':8,'Sep':9,'Oct':10,'Nov':11,'Dec':12 }
df_test['first_payment_date']=df_test['first_payment_date'].apply(dateformatchange)

In [5]:
import datetime

now = datetime.datetime.now()
today=now.strftime("%Y-%m-%d")
df['loan_age_months'] =  (pd.to_datetime(today)-pd.to_datetime(df.origination_date))/np.timedelta64(1,'M')
df['loan_age_months'] = df['loan_age_months'].astype(int)
df['remaining_months']=df['loan_term']-df['loan_age_months']

In [6]:
df_test['loan_age_months'] =  (pd.to_datetime(today)-pd.to_datetime(df_test.origination_date))/np.timedelta64(1,'M')
df_test['loan_age_months'] = df_test['loan_age_months'].astype(int)
df_test['remaining_months']=df_test['loan_term']-df_test['loan_age_months']

In [7]:
df['mothly_amount']=(df['unpaid_principal_bal']/df['remaining_months'])

In [8]:
df_test['mothly_amount']=(df_test['unpaid_principal_bal']/df_test['remaining_months'])

In [9]:
#df['flagm']= df.apply(lambda row: checkmvalue(row), axis = 1)
#df_test['flagm']= df_test.apply(lambda row: checkmvalue(row), axis = 1) 

In [10]:
#df['flagmmissedno']= df.apply(lambda row: checknummissed(row), axis = 1)
#df_test['flagmmissedno']= df_test.apply(lambda row: checknummissed(row), axis = 1)

In [11]:
#df['flagmmaxmissed']= df.apply(lambda row: checkmaxmissed(row), axis = 1)
#df_test['flagmmaxmissed']= df_test.apply(lambda row: checkmaxmissed(row), axis = 1)

In [12]:
X=df.drop('m13',axis=1)
y=df['m13']


In [13]:
X, cat_cols = one_hot_encoder(X, False)
df, cat_cols = one_hot_encoder(df, False)
df_test, cat_cols = one_hot_encoder(df_test, False)

In [14]:
X=X.drop(['m1','m2','m3','m4','m5','m6','m7','m8','m9','m10','m11','insurance_type'],axis=1)
df_test=df_test.drop(['m1','m2','m3','m4','m5','m6','m7','m8','m9','m10','m11','insurance_type'],axis=1)

In [45]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 101)

# Train the model


In [46]:
regr = RandomForestClassifier(n_estimators = 10, max_depth = 10, random_state = 101)

In [47]:
# Import GridSearchCV
from sklearn.model_selection import GridSearchCV

# Find the best parameters for the model
parameters = {
    'max_depth': [90,110],
    'n_estimators': [1400],
    'min_samples_split' : [5],
# Minimum number of samples required at each leaf node
    'min_samples_leaf' : [ 2],
# Method of selecting samples for training each tree
     'bootstrap' : [False]
}
gridforest = GridSearchCV(regr, parameters, cv = 3, n_jobs = -1, verbose = 10)
gridforest.fit(X_train, y_train)
gridforest.best_params_

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Done   2 out of   6 | elapsed:  4.1min remaining:  8.2min
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:  4.2min remaining:  4.2min
[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:  4.2min remaining:  2.1min
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  4.3min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  4.3min finished


{'bootstrap': False,
 'max_depth': 90,
 'min_samples_leaf': 2,
 'min_samples_split': 5,
 'n_estimators': 1400}

In [15]:
from sklearn import svm, grid_search

C:\Users\kaushik\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\kaushik\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [ ]:

clfsvc=svm.SVC()
clfsvc.fit(X,y)

In [50]:
pred=gridforest.predict(df_test)

In [51]:
gridsearch.best_estimator_

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.65,
        importance_type='split', learning_rate=0.0085, max_bin=512,
        max_depth=-1, metric='binary_error', min_child_samples=5,
        min_child_weight=1, min_split_gain=0.5, n_estimators=36, n_jobs=-1,
        nthread=5, num_class=1, num_leaves=27, objective='binary',
        random_state=None, reg_alpha=1.2, reg_lambda=1.2,
        scale_pos_weight=1, silent=True, subsample=0.7,
        subsample_for_bin=200, subsample_freq=1)

In [52]:
pred[pred==1]

array([], dtype=int64)

In [24]:
sub=pd.DataFrame(columns=['loan_id','m13'])
sub['loan_id']=testmain['loan_id']
sub['m13']=pred
sub.to_csv("C:\\Users\kaushik\Desktop\ML\independence day hackathon\\sub.csv",index=False)

In [23]:
sub.drop(['index'],axis=1).

SyntaxError: invalid syntax (<ipython-input-23-e6a08769f3f2>, line 1)